In [ ]:
import pandas as pd
import numpy as np
from pybaseball import statcast
from datetime import datetime
import os
import pickle

In [ ]:
#You can run this notebook for any year, just be sure to change the dates in this cell

#get statcast data
#data_15 = statcast("2015-01-01","2015-12-31")
#data_16 = statcast("2016-01-01","2016-12-31")
#data_17 = statcast("2017-01-01","2017-12-31")
#data_18 = statcast("2018-01-01","2018-12-31")
#data_19 = statcast("2019-01-01","2019-12-31")
#data_21 = statcast("2021-01-01","2021-12-31")





In [ ]:
eco = {'0-0-0':0.0,'1-0-0':0.0,'0-1-0':0.0,'2-0-0':0.0,'0-2-0':0.0,'1-1-0':0.0
                      ,'3-0-0':0.0,'2-1-0':0.0,'1-2-0':0.0,'2-2-0':0.0,'3-1-0':0.0,'3-2-0':0.0,
                      '0-0-1':0.0,'1-0-1':0.0,'0-1-1':0.0,'2-0-1':0.0,'0-2-1':0.0,'1-1-1':0.0
                      ,'3-0-1':0.0,'2-1-1':0.0,'1-2-1':0.0,'2-2-1':0.0,'3-1-1':0.0,'3-2-1':0.0,
                      '0-0-2':0.0,'1-0-2':0.0,'0-1-2':0.0,'2-0-2':0.0,'0-2-2':0.0,'1-1-2':0.0
                      ,'3-0-2':0.0,'2-1-2':0.0,'1-2-2':0.0,'2-2-2':0.0,'3-1-2':0.0,'3-2-2':0.0}
    
class RunValueMatrix:     
    
    def __init__(self):
        self.occurances = {'0-0-0':eco.copy(),'0-0-1':eco.copy(),'0-1-0':eco.copy(),'1-0-0':eco.copy(),'0-1-1':eco.copy(),'1-0-1':eco.copy(),'1-1-0':eco.copy(),'1-1-1':eco.copy()}
        self.runs_to_end = {'0-0-0':eco.copy(),'0-0-1':eco.copy(),'0-1-0':eco.copy(),'1-0-0':eco.copy(),'0-1-1':eco.copy(),'1-0-1':eco.copy(),'1-1-0':eco.copy(),'1-1-1':eco.copy()}
        self.run_expectency = {'0-0-0':eco.copy(),'0-0-1':eco.copy(),'0-1-0':eco.copy(),'1-0-0':eco.copy(),'0-1-1':eco.copy(),'1-0-1':eco.copy(),'1-1-0':eco.copy(),'1-1-1':eco.copy()}
    
        
    
    def update_occ_runs(self,base,count_outs,runs):
        self.occurances[base][count_outs] = self.occurances[base][count_outs] +  1.0 
        self.runs_to_end[base][count_outs] = self.runs_to_end[base][count_outs] + float(runs)
        self.run_expectency[base][count_outs] = self.runs_to_end[base][count_outs]/self.occurances[base][count_outs]
        
    
    def get_rexp_frame(self):
        s = pd.DataFrame.from_records(data=[self.run_expectency[col] for col in self.run_expectency.keys()])
        s['base_state'] = self.run_expectency.keys()
        s.set_index('base_state', inplace=True)
        self.run_expectency_dataframe = s
        return s
        

In [ ]:
def is_nan(val):
    return 0 if pd.isna(val) else 1



In [ ]:
def score_game_re288(game,runvaluematrix):
    t = game[['inning', 'inning_topbot']].drop_duplicates().sort_values(by=['inning', 'inning_topbot'], ascending =[True,False]).reset_index(drop=True)
    for i in t.index:
        inning_frame = game.loc[(game['inning'] == t.iloc[i]['inning']) & 
                                  (game['inning_topbot'] == t.iloc[i]['inning_topbot'])].sort_values(by=['at_bat_number', 'pitch_number'], ascending =[True,True]).reset_index(drop = True)
        for j in inning_frame.index:
            count_out = f"{inning_frame.iloc[j]['balls']}-{inning_frame.iloc[j]['strikes']}-{inning_frame.iloc[j]['outs_when_up']}"
            base = f"{is_nan(inning_frame.iloc[j]['on_3b'])}-{is_nan(inning_frame.iloc[j]['on_2b'])}-{is_nan(inning_frame.iloc[j]['on_1b'])}"
            runs_to_end = max(inning_frame.loc[inning_frame.index >= j]['bat_score']) - inning_frame.iloc[ j]['bat_score']
            try:
                runvaluematrix.update_occ_runs(base,count_out,runs_to_end)
            except KeyError as e:
                print(e)
                continue
                
            
def score_year_re288(year,matrix):
    i = 0
    game_pks = pd.unique(year.game_pk)
    for game in game_pks:
        i+=1 
        if round(i/len(game_pks) * 100) % 20 == 0:
            print(i/len(game_pks) * 100)
        game_frame = year.loc[year['game_pk']==game]
        score_game_re288(game_frame,matrix)


    

In [ ]:
def get_year_run_exp(year):
    data = statcast(f"{year}-01-01",f"{year}-12-31")
    data = pd.DataFrame(data)[['game_pk','bat_score','balls', 'strikes','outs_when_up','on_3b', 'on_2b', 'on_1b','at_bat_number','pitch_number','inning', 'inning_topbot']]
    matrix = RunValueMatrix()
    score_year_re288(data,matrix)
    return matrix

In [ ]:
run_exp ={}
for x in [2015,2016,2017,2018,2019,2021]:
    run_exp[x] = get_year_run_exp(x)
    
#this took a while to run, so I ran it overnight and saved the objects in a pickled file to pick back up later
with open('run_expectency.txt','wb') as retext:
    pickle.dump(run_exp,retext)
    

    
    

In [ ]:
#a day later and here we unpickle our progress. Had a realization that I just want the dataframes soooooo....
unpickle = pickle.load(open('run_expectency.txt','rb'))
for year in  [2015,2016,2017,2018,2019,2021]:
    unpickle[year] = unpickle[year].get_rexp_frame()
    
    
with open('run_expectency_frames.txt','wb') as retext:
    pickle.dump(unpickle,retext)


In [ ]:
frame_pickled= pickle.load(open('run_expectency_frames.txt','rb'))
frame_pickled[2021].style